In [3]:
!nvidia-smi

Sun Jun 26 00:00:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.25       Driver Version: 516.25       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000   WDDM  | 00000000:65:00.0 Off |                    0 |
| 30%   28C    P8     8W / 230W |      0MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A5000   WDDM  | 00000000:B3:00.0 Off |                    0 |
| 30%   

# CCKS2022基于知识图谱的优质文章识别

In [ ]:
!set CUDA_VISIBLE_DEVICES=0,1
!python ccks2022_classifier.py \
  --task_name=text-clf \
  --do_train \
  --do_eval \
  --data_dir=data/ccks2022/ \
  --bert_model=pretrained_models/nezha-base-wwm \
  --max_seq_length=512 \
  --train_batch_size=28 \
  --eval_batch_size=28 \
  --learning_rate=1e-4  \
  --num_train_epochs=5.0 \
  --output_dir=output/ccks2022/2/
  # --warmup_proportion=0.2

In [ ]:
!python ccks2022_pred.py \
  --data_dir=data/ccks2022/ \
  --trained_model_dir=trained_models/ccks2022/ \
  --output_dir=output/ccks2022/2/predict/ \
  --eval_batch_size=30 \
  --max_seq_length=512

## 输出结果

In [ ]:
import json
import pandas as pd

df = pd.read_json('data/ccks2022/process/test.unlabel.json', encoding='utf8', lines=True)
labels = pd.read_csv("test_results.txt", encoding='utf8', header=None, names=['labels'])
s = ""
for i in range(df.shape[0]):
    dic = dict()
    dic['url'] = df.iloc[i, :]["url"]
    dic['label'] = int(labels.iloc[i, :]["labels"])
    s += json.dumps(dic, ensure_ascii=False) + "\n"
    if i == 0:
        print(s[:100])
with open("data/ccks2022/process/result/result.txt", encoding="utf-8", mode='w') as f:
    f.write(s)
    f.close()

# 继续预训练

In [6]:
!set CUDA_VISIBLE_DEVICES=0,1
!python ccks2022_for_pretraining.py \
  --task_name=text-clf \
  --do_train \
  --data_dir=data/ccks2022/ \
  --bert_model=pretrained_models/nezha-cn-base \
  --max_seq_length=512 \
  --train_batch_size=8 \
  --eval_batch_size=8 \
  --learning_rate=3e-5  \
  --num_train_epochs=5.0 \
  --output_dir=output/ccks2022/continue_pretraining/

init model...
missing keys:[]
unexpected keys:[]
error msgs:[]
Total param num：160336522
missing keys:['bert.ptm.embeddings.word_embeddings.weight', 'bert.ptm.embeddings.token_type_embeddings.weight', 'bert.ptm.embeddings.LayerNorm.weight', 'bert.ptm.embeddings.LayerNorm.bias', 'bert.ptm.encoder.layer.0.attention.self.query.weight', 'bert.ptm.encoder.layer.0.attention.self.query.bias', 'bert.ptm.encoder.layer.0.attention.self.key.weight', 'bert.ptm.encoder.layer.0.attention.self.key.bias', 'bert.ptm.encoder.layer.0.attention.self.value.weight', 'bert.ptm.encoder.layer.0.attention.self.value.bias', 'bert.ptm.encoder.layer.0.attention.output.dense.weight', 'bert.ptm.encoder.layer.0.attention.output.dense.bias', 'bert.ptm.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.ptm.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.ptm.encoder.layer.0.intermediate.dense.weight', 'bert.ptm.encoder.layer.0.intermediate.dense.bias', 'bert.ptm.encoder.layer.0.output.dense.weight', 'bert


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/916 [00:00<?, ?it/s]C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cuda\Indexing.cu:975: block: [193,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cuda\Indexing.cu:975: block: [193,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cuda\Indexing.cu:975: block: [193,0,0], thread: [2,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cuda\Indexing.cu:975: block: [193,0,0], thread: [3,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cuda\Indexing.cu:975: block: [193,0,0], thread: [4,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cuda\Indexing.cu:975: block: [193,0,0], thread: [5,0,0] Ass